# Settings

In [ ]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
print(sep_local)

In [ ]:
print(os.getcwd())

In [ ]:
import sys
os.chdir('..' + sep_local +'..' + sep_local +'..' + sep_local + '..' + sep_local + '..' + sep_local + '..') # For Linux import
print(os.getcwd())

In [ ]:
import tensorflow as tf
print(tf.__version__)

# Dataset loading

In [ ]:
dataset_name='CBSD68'

In [ ]:
images_dir =  '.' + sep_local + 'data' + sep_local + '.CBSD68'
validation_percentage = 10
valid_format = 'png'

In [ ]:
images_dir

In [ ]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [ ]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

In [ ]:
from PIL import Image
trace_image = Image.open(images_dir+sep_local+'original'+sep_local+'{:04d}.png'.format(66))

In [ ]:
trace_image

In [ ]:
#image = np.asarray(trace_image)/255.0
#timage = put_random_text(image)
#Image.fromarray((timage * 255.0).astype(np.uint8), mode='RGB')


In [ ]:
image_size_original=(481, 321, 3)
scale = 2
image_size = list(map(lambda x: x//scale , image_size_original[:-1])) + [image_size_original[-1]]
image_size = (*image_size,)
batch_size = 16
latents_dim = 150
intermediate_dim = 50

In [ ]:
image_size

In [ ]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode=None
)

## input is half of the output

In [ ]:
scale=1
inputs_shape = list(map(lambda x: x//scale , image_size[:-1])) + [image_size[-1]]
inputs_shape = (*inputs_shape, )

In [ ]:
image_size, inputs_shape

In [ ]:
shrink_fn = lambda image: tf.image.resize(image, inputs_shape[:-1])
enlarge_fn = lambda image: tf.image.resize(image, image_size[:-1])

In [ ]:
import numpy as np
import cv2

In [ ]:
import random
import string
characters = string.ascii_letters+string.ascii_letters+string.ascii_letters+string.digits+string.punctuation
word_len = range(10)
def random_sentance_generator():
    sentance = ''
    rn = random.choice(range(1, 20))
    for i in range(rn):
        sentance = sentance + ' ' + ''.join(random.choice(characters) for i in word_len)

    return sentance

In [ ]:
random_sentance_generator()

In [ ]:
fonts = list(set([eval('cv2.{}'.format(i)) for i in dir(cv2) if i.startswith('FONT_')]))
def put_random_text(image):

    image_dim = image.shape[:-1]
    #scale=1
    image_cv = cv2.cvtColor((image * 255).astype(np.uint8), cv2.IMREAD_COLOR)
    #image_cv = cv2.resize( image_cv, (image_dim[0]*scale, image_dim[1]*scale))
                          
    for _ in range(5):
        x_loc = random.choice(range(image_cv.shape[0]))
        y_loc = random.choice(range(image_cv.shape[1]))
        loc = (x_loc, y_loc)


        cv2.putText(img=image_cv, 
                    text=random_sentance_generator(), 
                    org=loc,
                    fontFace=random.choice(fonts),
                    fontScale=random.choice(range(1, 3)),
                    color=(random.choice(range(256)), random.choice(range(256)), random.choice(range(256))),
                    thickness=random.choice(range(3))
                   )
                
    image_cv = np.asarray(Image.fromarray(np.asarray(cv2.resize(image_cv, image_dim)).astype(np.uint8), mode='RGB')\
                          .rotate(90, expand=True))/255.0

    return image_cv

In [ ]:
def put_text_fn(images):
    if len(images.shape)<4:
        images = [images]
    return np.array([put_random_text(image) for image in images])

In [ ]:

def generator_text_putter(generator): 
    while True:
        batch = next(generator)
        yield  put_text_fn(batch), batch


In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: generator_text_putter(training_generator), 
    output_types= (tf.float32, tf.float32),
    output_shapes=(tf.TensorShape((batch_size, ) +  inputs_shape), tf.TensorShape((batch_size, ) + image_size)),
)

test_ds = tf.data.Dataset.from_generator(
    lambda: generator_text_putter(testing_generator), 
    output_types= (tf.float32, tf.float32),
    output_shapes=(tf.TensorShape((batch_size, ) +  inputs_shape), tf.TensorShape((batch_size, ) + image_size)),
)



In [ ]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [ ]:
_instance_scale

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(image_size, Iterable):
    _outputs_shape = np.prod(image_size)
_outputs_shape    

# Model's Layers definition

In [ ]:
kernel_size=3
stride = 2

In [ ]:
c = list(map(lambda x: x// (stride*stride), image_size[:-1]))
c = (*c, intermediate_dim)
c

In [ ]:
enc_lays = [
    tf.keras.layers.UpSampling2D(size=(2, 2)),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product(c), activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=c),
    tf.keras.layers.Conv2DTranspose(filters=intermediate_dim, kernel_size=kernel_size, strides=(stride, stride), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=intermediate_dim, kernel_size=kernel_size, strides=(stride, stride), padding="SAME", activation='relu'),
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=kernel_size, strides=(1, 1), padding="SAME")
]

# Model definition

In [ ]:
model_name = dataset_name+'_Conv_Pre_Upsampling_coloring_Grayscale'
#windows
#experiments_dir='..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+sep_local+'experiments'+sep_local + model_name

#linux 
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [ ]:
variables_params = \
[
    {
        'name': 'inference',  #'upsampler',
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', #'constructive', 
        'inputs_shape':latents_dim,
        'outputs_shape':image_size,
        'layers':dec_lays
    }
]

In [ ]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

In [ ]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None,#to restore trained model, set filepath=_restore
    episode_len=1
    )

In [ ]:
image_size, inputs_shape

In [ ]:
#ae.compile(metrics=None)

#ae.compile(metrics=create_metrics())
ae.compile()

# Callbacks

In [ ]:
# added for linux warning suppression
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)


from training.callbacks.trace_image_reconstruction import trace_reconstruction
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_reconstuction_dir = os.path.join(experiments_dir, 'image_reconstuction_dir')
create_if_not_exist(image_reconstuction_dir)
absolute = abspath(image_reconstuction_dir)
print("image_reconstuction_dir",absolute)

In [ ]:
image = put_text_fn(shrink_fn(np.asarray(trace_image)).numpy()/255.0)[0]
img_reconst = trace_reconstruction(filepath=image_reconstuction_dir, image=image, gen_freq=5)

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=int(1e4),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, img_reconst],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=int(1e4)
)